In [1]:
import requests
import traceback
import os
import boto3
import pandas as pd
from dotenv import load_dotenv
from github_python_fetch import fetch_function
from query_package import get_query_package
from datetime import datetime


# GitHub Access 
token = os.getenv('GITHUBTOKEN')

# AWS Access
access = os.getenv('ACCESS')
secret = os.getenv('SECRET')

# RDS Access
username = os.getenv('USER')
password = os.getenv('PASS')
server = os.getenv('SERVER')
db = os.getenv('DB')


# Access RDS Functions via GitHub
rds_connector_url = 'https://raw.githubusercontent.com/CharlesRoss-MBakerIntl/Tidal-Basin-Functions/main/rds_connector.py' # Set url to python file of github
rds_connector = fetch_function(rds_connector_url, token) # Pull function from github using requests
exec(rds_connector) # Execute the file


# Access RDS Functions via GitHub
s3_manager_url = 'https://raw.githubusercontent.com/CharlesRoss-MBakerIntl/Tidal-Basin-Functions/main/rds_connector.py' # Set url to python file of github
s3_manager = fetch_function(s3_manager_url, token) # Pull function from github using requests
exec(s3_manager) # Execute the file


# Pull Query Package from File
query_package = get_query_package()


# Store RDS Functions Locally
conn, cursor = rds_connection(username, password, db, server) # Connect to RDS Database
rds = RDS(conn, cursor, query_package) # Create Instance of RDS Table
df = rds.df

Exception: Failed to Connect to RDS Database: connection to server at "tidalbasin-cdbg-dr-fl-ida-backendauroraclusterclus-mopwhclky2lb.cluster-ro-ce8xd2bu2edx.us-east-2.rds.amazonaws.com" (10.0.136.135), port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?
